In [1]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


## Pretrained API function name_emb

In [2]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
# api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [3]:
whole_dict ={}
whole_dict_count ={}
pretrain_dict ={}
whole_dict = whole_dict.fromkeys(api_li,np.array([0.0]*768))
pretrain_dict = pretrain_dict.fromkeys(api_li,np.array([0.0]*768))
whole_dict_count = whole_dict.fromkeys(api_li,0)

encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles-partial/encode_dict.pkl','wb'),obj=encode_dict)
# encode_dict

whole_dict['GetUrlCacheEntryInfo'] , whole_dict_count

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [4]:
root_dir = './data/tree-rep-profiles_one2many/normal/'

# whole_dict = {}
 
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith("_clsfeature.pkl"), hl_list))
        for pkl in hl_list:
            temp = pickle.load(open(pkl,'rb'))
            path2 = pkl.replace('_clsfeature.pkl','.profile_clean')
            with open(path2,encoding='ISO 8859-1') as f:
                lines = f.read().splitlines()
            assert len(temp) == len(lines)
            for num,line in zip(temp,lines):
                api = line.split(' ')[0]
#                 if api == 'GetUrlCacheEntryInfo':
#                     print(api)
                whole_dict[api] = whole_dict[api]+num
                whole_dict_count[api] = whole_dict_count[api] + 1
whole_dict_count , whole_dict['LoadLibrary']

100%|██████████| 184/184 [00:09<00:00, 19.41it/s]


({'LoadLibrary': 245446,
  'CreateProcess': 3660,
  'OpenProcess': 11942,
  'ExitProcess': 4013,
  'TerminateProcess': 32,
  'WinExec': 359,
  'CreateRemoteThread': 9445,
  'CreateThread': 8673,
  'CopyFile': 50250,
  'CreateFile': 477930,
  'DeleteFile': 6573,
  'RegSetValue': 59666,
  'RegCreateKey': 92317,
  'RegDeleteKey': 669,
  'RegDeleteValue': 4760,
  'RegQueryValue': 1149992,
  'RegEnumValue': 163194,
  'WinHttpConnect': 2054,
  'WinHttpOpen': 2059,
  'WinHttpOpenRequest': 2054,
  'WinHttpReadData': 2024,
  'WinHttpSendRequest': 2058,
  'InternetOpen': 870,
  'InternetConnect': 824,
  'HttpSendRequest': 801,
  'GetUrlCacheEntryInfo': 12},
 array([ 9.99023539e+03, -4.64189094e+05,  2.79185544e+05,  4.57839031e+05,
         8.98026236e+05,  5.56751468e+05, -1.95441824e+05, -5.62586966e+05,
        -3.73815600e+05, -2.88895406e+05,  5.98319258e+05, -3.14612828e+05,
        -3.19104115e+05, -4.91197893e+05, -1.40438498e+06, -1.82591745e+05,
         4.35847304e+05,  7.41994294e+05

In [5]:
for api in api_li:
    pretrain_dict[api] = whole_dict[api] / (whole_dict_count[api] + 1e-8)
pretrain_dict[api]

array([ 2.90780425e+00, -1.14000314e+00, -1.23680325e+00,  3.70944555e+00,
        2.96724180e+00,  1.18944148e+00,  3.81189545e-01,  4.10654346e-01,
       -1.89083159e+00, -5.29230817e+00,  6.13632976e-01,  2.01550692e-01,
       -1.76765547e+00, -1.22828618e+00, -2.96636404e+00,  3.60461453e-01,
        2.66539323e+00,  2.28999640e+00,  2.57476166e+00,  1.93482054e+00,
       -3.53761593e+00,  1.68603829e+00,  2.68108173e+00,  4.32407077e+00,
       -2.18295872e+00, -2.90814884e+00, -9.98997519e-01,  3.29604331e+00,
       -2.55896970e-01, -1.91090130e+00, -3.51309354e-02,  2.35916932e+00,
       -2.67268781e+00, -1.75706307e-01, -1.26416971e+00,  2.74977513e+00,
       -6.93236319e+00,  2.12861117e+00, -1.21628312e+00,  2.16459071e+00,
        1.84460691e+00,  2.02197528e+00, -4.32961603e-01, -2.04998185e+00,
        8.30465589e-01,  3.71859781e+00, -2.47195814e+00, -2.70330389e-01,
        1.86726554e+00,  3.41869942e+00,  7.76047239e-01, -4.07282356e+00,
        3.15147126e-01, -

In [6]:
embedding_matrix = np.zeros((len(pretrain_dict) + 1, 768))
for word, i in encode_dict.items():
    embedding_vector = pretrain_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
pickle.dump(file=open('data/tree-rep-profiles-partial/api_emb_matrix.pkl','wb'),obj=embedding_matrix)
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04070238, -1.89120659,  1.13746219, ...,  1.66638085,
         1.35397944, -1.75575362],
       [ 1.89995008, -1.14056988,  2.28380203, ...,  2.87513128,
        -0.15888875, -2.55449358],
       ...,
       [-0.09921578, -1.3831512 ,  1.06511487, ...,  5.01848707,
        -1.44074326,  0.17810244],
       [ 2.39957536, -0.60927182,  0.30928497, ...,  4.26993283,
         2.38004934, -0.26339761],
       [ 2.90780425, -1.14000314, -1.23680325, ...,  2.17426304,
         0.08331919, -1.01571888]])

# Prepare data vectors

In [7]:
all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
av_col = all_df.columns[1:].tolist()
root_dir = './data/tree-rep-profiles-partial/normal/'
fam_dir = next(os.walk(root_dir))[1]
fam_dir = [x.split('_')[0] for x in fam_dir]
av_df_col = []
for name in av_col:
    if name in fam_dir:
        av_df_col.append(name)
# av_df_col = ['profile'] + av_df_col
print(len(av_df_col))
av_df_col

44


['3.kazy',
 '4.zbot',
 '6.allaple',
 '11.virtob',
 '12.loadmoney',
 '13.graftor',
 '14.vobfus',
 '17.expiro',
 '18.sytro',
 '21.ramnit',
 '22.domaiq',
 '27.softpulse',
 '29.delf',
 '30.mplug',
 '32.browsefox',
 '33.autoit',
 '37.parite',
 '38.msil',
 '41.outbrowse',
 '42.sirefef',
 '43.mira',
 '44.screensaver',
 '45.firseria',
 '47.somoto',
 '51.shodi',
 '54.soltern',
 '55.zygug',
 '59.megasearch',
 '60.conjar',
 '63.jorik',
 '64.installerex',
 '65.shipup',
 '66.bdmj',
 '71.eggnog',
 '75.fesber',
 '78.vilsel',
 '81.picsys',
 '84.directdow',
 '88.clickdownload',
 '103.yantai',
 '104.mresmon',
 '111.vbran',
 '124.avmh',
 '151.downloadadmin']

In [8]:
samples_pid_train = pickle.load(file=open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(file=open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(file=open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))


In [9]:
def create_fam_data(root_dir,samples_pid,av_df_col=av_df_col):
# root_dir = './data/tree-rep-profiles-partial/normal/'
# samples_pid = samples_pid_train
    name_df = pd.DataFrame(0,columns=av_df_col,index=samples_pid)
    for pid in tqdm(samples_pid):
        paths = glob.glob(root_dir +'*/*/'+pid )
        name_li = []
        for path in paths:
            name_li.append(path.split('/')[4].split('_')[0])
        name_df.loc[pid,list(set(name_li))] = 1
    return name_df



In [10]:
root_dir = './data/tree-rep-profiles-partial/normal/'
train_fam_df = create_fam_data(root_dir,samples_pid_train)
root_dir = './data/tree-rep-profiles-partial/DEV/'
valid_fam_df = create_fam_data(root_dir,samples_pid_valid)
root_dir = './data/tree-rep-profiles-partial/TEST/'
test_fam_df = create_fam_data(root_dir,samples_pid_test)

100%|██████████| 264/264 [00:00<00:00, 293.10it/s]


In [11]:
dil= r"[@#]+\ *" #等號、底線被保留
def byterep_ans(byterep_list,max_length):
    '''
    Input: byte sequenxe (list of pickles path) 
    '''
    rep_list=[]
    for byterep in byterep_list:
        rep = pickle.load(open(byterep,'rb'))
        if len(rep) < max_length:
            for _ in range(max_length-len(rep)):
                rep.append(0)
        rep_list.append(rep)
    return rep_list

# 寫不同模組 來處理x2 y1 y2
def Sent2vec_emb(root_dir,all_df,max_length,dim=768,encode_dict=encode_dict,av_df_col=av_df_col):
    '''
    Create sent2vec profile embedding
    Input: root_dir=>fam-tree-profile
    Output: 
    '''
    sent_pad = [0]*dim
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            byterep_list = [x.split('.')[0]+'_byterep.pickle' for x in hl_list]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            byterep_list = [os.path.join(in_directory, f) for f in byterep_list]
            rep_list = byterep_ans(byterep_list,max_length)
            for profile in hl_list:
                pid = profile.split('/')[-1]
#                 print(in_directory,pid)
                profile_df = all_df[all_df.index == pid]
                fam_ans = list(profile_df.values[0])

                profile_emb = []
                func_name_emb = []
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0]
                    if func_name not in api_li:
                        print('=ERROR:=',profile,'=>',temp)
                    func_name_emb.append(encode_dict[func_name])
#                     emb = model.embed_sentence(temp)
#                     emb = emb[0]
#                 try:
                profile_emb = list(pickle.load(open(profile.replace('.profile','_tokenfeature.pkl'),'rb')))
#                 except FileNotFoundError:
                    
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                all_profiles.append(np.array(profile_emb))
                all_api_name.append(func_name_emb)
                all_fam_ans.append(fam_ans)
            all_byterep.extend(np.array(rep_list))
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0)
    return np.array(all_profiles) , np.array(all_api_name) , np.array(all_fam_ans) , np.array(all_byterep)

In [13]:
max_length = 213

root_dir = './data/tree-rep-profiles-partial/normal/'
train_emb, train_emb_api, train_fam_ans, train_rep_ans = Sent2vec_emb(root_dir,train_fam_df,max_length=max_length)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/DEV/'
valid_emb, valid_emb_api, valid_fam_ans, valid_rep_ans = Sent2vec_emb(root_dir,valid_fam_df,max_length)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/TEST/'
test_emb, test_emb_api, test_fam_ans, test_rep_ans = Sent2vec_emb(root_dir,test_fam_df,max_length)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

  7%|▋         | 3/44 [00:00<00:01, 22.89it/s]

train of sent2vec vector: (11141, 213, 768) (11141, 213) (11141, 44) (11141, 213)


  7%|▋         | 3/44 [00:00<00:02, 17.90it/s]

valid of sent2vec vector: (437, 213, 768) (437, 213) (437, 44) (437, 213)


100%|██████████| 44/44 [00:06<00:00,  6.74it/s]

test of sent2vec vector: (424, 213, 768) (424, 213) (424, 44) (424, 213)


In [15]:
pickle.dump(file=open('data/tree-rep-profiles-partial/TRAIN_vec.pkl','wb'),protocol=4,obj=(train_emb, train_emb_api, train_fam_ans, train_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/DEV_vec.pkl','wb'),protocol=4,obj=(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/TEST_vec.pkl','wb'),protocol=4,obj=(test_emb, test_emb_api,test_fam_ans,test_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/fam_df.pkl','wb'),protocol=4,obj=(train_fam_df, valid_fam_df,test_fam_df))